In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 8
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 76s 433ms/step - accuracy: 0.6736 - loss: 1.2543 - val_accuracy: 0.8564 - val_loss: 0.3953
Epoch 2/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.8975 - loss: 0.2976 - val_accuracy: 0.8317 - val_loss: 0.5359
Epoch 3/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 128ms/step - accuracy: 0.8729 - loss: 0.4050 - val_accuracy: 0.9208 - val_loss: 0.2273
Epoch 4/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - accuracy: 0.9253 - loss: 0.1881 - val_accuracy: 0.9208 - val_loss: 0.2151
Epoch 5/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.9256 - loss: 0.2197 - val_accuracy: 0.9109 - val_loss: 0.2250
Epoch 6/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 128ms/step - accuracy: 0.9049 - loss: 0.2798 - val_accuracy: 0.9406 - val_loss: 0.1874
Epoch 7/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 14s 116ms/step - accuracy: 0.9363 - loss: 0.1587 - val_accuracy: 0.9059 - val_loss: 0.2468
Epoch 8/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - accuracy: 0.9602 - loss: 0

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

26/26 ━━━━━━━━━━━━━━━━━━━━ 27s 446ms/step - accuracy: 0.9620 - loss: 0.1404

Test Loss: 0.1853
Test Accuracy: 95.61%
26/26 ━━━━━━━━━━━━━━━━━━━━ 24s 525ms/step
Accuracy: 95.60975609756098
Precision: 95.70583887657058
Recall: 95.60975609756098
F1 Score: 95.58693051081455

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.9091    0.9804    0.9434        51
day-cap-ri-set     0.9649    0.9649    0.9649        57
        ri-set     0.9583    0.8846    0.9200        52
       vet-nut     1.0000    1.0000    1.0000        45

      accuracy                         0.9561       205
     macro avg     0.9581    0.9575    0.9571       205
  weighted avg     0.9571    0.9561    0.9559       205


Accuracy for each class:
bong-troc-son: 0.98
day-cap-ri-set: 0.96
ri-set: 0.88
vet-nut: 1.00
